# Gillespie Algorithm for a SIR Model with Wise and Risky Indiviuals
### by Luca Sbano (Theory & Implementation) and Steffen Bauer (Implementation & Jupyter)
***

__Definitions:__
<br>

|Variable | Meaning                                             |
|---------|-----------------------------------------------------|
|$W$:     | susceptible and wise                                |
|$R$:     | susceptible and risky                               |
|$I$:     | infectious                                          |
|$D$:     | dead                                                |
|$S$:     | individuals cured and who can no longer be infected |

__Reactions:__
<br>
$$
\begin{align}
    W &\rightarrow^a R\\
    R &\rightarrow^{\alpha} W\\
    R + I &\rightarrow^c 2I\\
    W + I &\rightarrow^b 2I\\
    I &\rightarrow^{\beta} W\\
    I &\rightarrow^d D\\
    I &\rightarrow^{\rho} S
\end{align}
$$

## Algorithm

>1. Generate two random numbers $r_1$, $r_2$ uniformly distributed in
$[0, 1]$.
2. Compute $\mathcal{R}_0 := \sum_{j=1}^r \mathcal{R}_j$.
3. Compute $\tau := \frac{1}{\mathcal{R}_0} \ln[\frac{1}{r_1}]$.
4. Set time of next rule execution to $t + \tau$.
5. Compute wich rule is executed at time $t + \tau$. Find $j$ such that
$$
\frac{1}{\mathcal{R}_0} \sum_{i=1}^{j-1} \mathcal{R}_i < r_2 \leq  \frac{1}{\mathcal{R}_0} \sum_{i=1}^{j} \mathcal{R}_i
$$
6. Execute rule $R_j$ and update to new system configuration.
7. Go to step 1 with updated time $t::= t + \tau$ (where $::=$ means replacement) if $t < T$, otherwise stop.

(If you want to see Daniel Gillespie presenting his algorithm in 2017, go to [youtube](https://youtu.be/atOc2v8Wtcw).)

## Implementation

First load some libraries:

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact
from IPython.display import display

Define the algorithm [above](#Algorithm) as a function (later used for interactivity):

In [4]:
def gillespie(rWiseToRisky     = 0.1, 
              rRiskyToWise     = 0.03, 
              rWiseToInfected  = 0.001, 
              rRiskyToInfected = 0.01,
              rCure            = 0.2,
              rFatalities      = 0.08,
              rRecover         =0.1
             ):
    '''
    Gillespie algorithm to solve SIR model for wise and risky people
    Implementation of Gillespie algorithm and SIR model: Luca Sbano
    Slight modifications for interactivity with slider : Steffen Bauer
    '''
    
    # Input parameters 
    N     = 200              #        int;   total population  
    T     = 100.0            #        float; maximum elapsed time 
    t     = 0.0              #        float; start time
    a     = rWiseToRisky     # (0.1)  float; rate of wise to risky
    alpha = rRiskyToWise     # (0.03) float; rate of risky to wise
    b     = rWiseToInfected  # (0.001)float; rate of wise to infected
    c     = rRiskyToInfected # (0.01) float; rate of risky to infected
    beta  = rCure            # (0.2)  float; rate of cure
    d     = rFatalities      # (0.08) float; rate of fatalities
    rho   = rRecover         # (0.1)  float; rate of recovering and getting immunity
    
    n_I   = 1                #        int;   initial Infected population
    n_D   = 0                #        int;   initial conditions on Dead people 
    n_S   = 0                #        int;   initial conditions on Saved people 
    n_R   = 50               #        int;   initial conditions on Risky people
    n_W   = N - n_I - n_R    #        int;   initial conditions on Wise people

    # Initialize results list
    MAXITER       = 10000
    SIR_data      = np.zeros((MAXITER, 6))
    SIR_data[0,:] = [t, n_W, n_I, n_R, n_S, n_D]

    it = 0
    # Main loop
    while t < T and it < MAXITER:
        if n_I == 0:
            break
        it += 1

        # Rates/Weights calculation
        w1 = a * n_W
        w2 = alpha * n_R
        w3 = c * n_I * n_R
        w4 = b * n_I * n_W
        w5 = beta * n_I
        w6 = d * n_I
        w7 = rho * n_I
        W  = w1 + w2 + w3 + w4 + w5 + w6 + w7
        
        # First uniformily distributed ranmdom number
        r_1 = np.random.uniform(0.0, 1.0)
        # Time increment
        dt  = -np.log(r_1) / W
        t   = t + dt

        #Second uniformily distributed ranmdom number
        r_2 = np.random.uniform(0.0, 1.0)
    
        #Choice of the reaction according to the probabilities
        if r_2 < w1 / W:
            n_W = n_W - 1
            n_R = n_R + 1
        if r_2>=w1/W and r_2 < (w1+w2) / W:
            n_R = n_R - 1
            n_W = n_W + 1
        if r_2>=(w1+w2)/W and r_2 < (w1+w2+w3)/W:
            n_R = n_R - 1
            n_I = n_I + 1
        if r_2>=(w1+w2+w3)/W and r_2 < (w1+w2+w3+w4)/W:
            n_W = n_W - 1
            n_I = n_I + 1
        if r_2>=(w1+w2+w3+w4)/W and r_2 < (w1+w2+w3+w4+w5)/W:
            n_I = n_I - 1
            n_W = n_W + 1
        if r_2>=(w1+w2+w3+w4+w5)/W and r_2< (w1+w2+w3+w4+w5+w6)/W:
            n_I = n_I - 1
            n_D = n_D + 1
        if r_2 >(w1+w2+w3+w4+w5+w6)/W:
            n_I = n_I - 1
            n_S = n_S + 1
        
        #print("{:f} \t {:d} \t {:d} \t {:d} \t {:d} \t {:d}".format(t, n_W, n_I, n_R, n_S, n_D))
        SIR_data[it, :] = [t, n_W, n_I, n_R, n_S, n_D]


    #Plotting
    SIR  = SIR_data[:it,:]
    time = SIR[:,0]
    nW   = SIR[:,1]
    nI   = SIR[:,2]
    nR   = SIR[:,3]
    nS   = SIR[:,4]
    nD   = SIR[:,5]

    fig, ax = plt.subplots(figsize=(15,5))
    line1,=ax.plot(time,nW, label='$n_W$')
    line2,=ax.plot(time,nI, label='$n_I$')
    line3,=ax.plot(time,nR, label='$n_R$')
    line4,=ax.plot(time,nS, label='$n_S$')
    line5,=ax.plot(time,nD, label='$n_D$')
    
    legend = ax.legend(bbox_to_anchor=(1.04,0.5), loc='center left', shadow=True, fontsize='x-large')
    ax.set_xlim(0., T)
    ax.set_ylim(0., N)
    
    plt.title('Simulation of a SIR model')
    plt.xlabel('time')
    plt.ylabel('Individuals')

    plt.show()

You could call the function simply by 
`gillespie()` or `gillespie(rWiseToRisky = 0.8)` if you want to chance the parameters / reaction rates.

But we can add interactivity by simply using `interact(function, paramters = (min, max, stepsize))`. And we get some lovely sliders.

_(Please, be patient and wait for updating of the graph.)_

In [6]:
interact(gillespie, rWiseToRisky     = (0.,1.,0.01), 
                    rRiskyToWise     = (0.,1.,0.01), 
                    rWiseToInfected  = (0.,1.,0.001),
                    rRiskyToInfected = (0.,1.,0.01),
                    rCure            = (0.,1.,0.01),
                    rFatalities      = (0.,1.,0.01),
                    rRecover         = (0.,1.,0.01))

interactive(children=(FloatSlider(value=0.1, description='rWiseToRisky', max=1.0, step=0.01), FloatSlider(valu…

<function __main__.gillespie(rWiseToRisky=0.1, rRiskyToWise=0.03, rWiseToInfected=0.001, rRiskyToInfected=0.01, rCure=0.2, rFatalities=0.08, rRecover=0.1)>

Now it's time to play with all these parameters. Choose them wisely!;-P